In [79]:
import numpy as np
import cv2

In [80]:
def downsample(image, pool_size, stride, pooling_function):
  """
  Generic downsampling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  pooling_function: function(block) -> pixel value
  """

  # Compute output size based on passed parameters
  in_height, in_width = image.shape
  out_height = (in_height - pool_size[0]) // stride[0] + 1
  out_width  = (in_width  - pool_size[1]) // stride[1] + 1

  # Construct empty output image
  output = np.zeros((out_height, out_width), dtype=image.dtype)

  # Iterate over every pixel in the output image
  for y in range(out_height):
    for x in range(out_width):

      # Compute the corresponding x section of the input image
      x_start = x * stride[1]
      x_end = x * stride[1] + pool_size[1]

      # Compute the corresponding y section of the input image
      y_start = y * stride[0]
      y_end = y * stride[0] + pool_size[0]

      # Construct the corresponding source block and compute the maximum value
      block = image[y_start:y_end, x_start:x_end]
      output[y,x] = pooling_function(block)

  return output

In [81]:
def max_pooling_downsample(image, pool_size=(2,2), stride=(2,2)):
  """
  Max pooling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  """

  # Define the max pooling function
  def max_pooling(block):
    return np.max(block)

  # Run downsampling with max pooling
  output = downsample(image, pool_size, stride, max_pooling)
  return output



def avg_pooling_downsample(image, pool_size=(2,2), stride=(2,2)):
  """
  Average pooling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  """

  # Define the avg pooling function
  def avg_pooling(block):
    return np.mean(block)

  # Run downsampling with avg pooling
  output = downsample(image, pool_size, stride, avg_pooling)
  return output


def med_pooling_downsample(image, pool_size=(2,2), stride=(2,2)):
  """
  Median pooling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  """

  # Define the med pooling function
  def med_pooling(block):
    return np.median(block)

  # Run downsampling with med pooling
  output = downsample(image, pool_size, stride, med_pooling)
  return output


In [82]:
def upsample(image, scale, interpolation_function):
  """
  Generic upsampling function.
  image: 2D numpy array (grayscale)
  scale: tuple (sh, sw)
  interpolation_function: function(image, src_y, src_x) -> pixel value
  """

  # Compute output image size
  in_height, in_width = image.shape
  out_height = in_height * scale[0]
  out_width  = in_width * scale[1]

  # Construct output image
  output = np.zeros((out_height, out_width), dtype=image.dtype)

  # Iterate over every pixel of the output image
  for y in range(out_height):
    for x in range(out_width):
      src_y = y / scale[0]
      src_x = x / scale[1]

      # Check if this pixel maps back to one of the original pixels
      if (src_y % 1 == 0 and src_x % 1 == 0):
        output[y, x] = image[int(src_y), int(src_x)]

      # If not, use interpolation to determine pixel value
      else:
        output[y, x] = interpolation_function(image, src_y, src_x)

  return output

In [83]:
def nn_interpolation_upsample(image, scale=(2,2)):
  """
  Upsampling function that uses nearest neighbour interpolation.
  image: 2D numpy array (grayscale)
  scale: tuple (sh, sw)
  """

  # Define the nearest neighbour interpolation function
  def nn_interpolation(image, src_y, src_x):
    """
    Nearest neighbour interpolation function.
    image: 2D numpy array (grayscale)
    src_x: float
    src_y: float
    """

    # Round to nearest integer coordinates
    closest_y = int(np.floor(src_y + 0.5))
    closest_x = int(np.floor(src_x + 0.5))

    # Ensure nearest integer coordinates are within image bounds
    closest_y = np.clip(closest_y, 0, image.shape[0] - 1)
    closest_x = np.clip(closest_x, 0, image.shape[1] - 1)

    return image[closest_y, closest_x]

  # Run upsampling using nearest neighbour interpolation
  return upsample(image, scale, nn_interpolation)


def bilinear_interpolation_upsample(image, scale=(2,2)):
  """
  Upsampling function that uses bilinear interpolation.
  image: 2D numpy array (grayscale)
  scale: tuple (sh, sw)
  """

  # Define the bilinear interpolation function
  def bilinear_interpolation(image, src_y, src_x):
    """
    Bilinear interpolation using averaging function.
    image: 2D numpy array (grayscale)
    src_x: float
    src_y: float
    """

    # Find surrounding coordinates in the original image (2x2 grid)
    y_min = int(np.floor(src_y))
    y_max = int(np.ceil(src_y))
    x_min = int(np.floor(src_x))
    x_max = int(np.ceil(src_x))

    # Ensure coordinates are within image bounds
    y_min = max(y_min, 0)
    y_max = min(y_max, image.shape[0] - 1)
    x_min = max(x_min, 0)
    x_max = min(x_max, image.shape[1] - 1)

    # Find surrounding pixel values
    top_left = image[y_min, x_min]
    top_right = image[y_min, x_max]
    bottom_left = image[y_max, x_min]
    bottom_right = image[y_max, x_max]

    # Compute distance to top_left surrounding pixel
    dy = src_y - y_min
    dx = src_x - x_min

    # Compute weighted average of surrounding pixel values
    weighted_average_value = (1 - dy) * (1 - dx) * top_left + \
                              (1 - dy) * dx * top_right + \
                              dy * (1 - dx) * bottom_left + \
                              dy * dx * bottom_right

    return weighted_average_value

  # Run upsampling using bilinear interpolation
  return upsample(image, scale, bilinear_interpolation)


def bicubic_interpolation_upsample(image, scale=(2,2)):
  """
  Upsampling function that uses bicubic interpolation.
  image: 2D numpy array (grayscale)
  scale: tuple (sh, sw)
  """

  # Define the cubic kernel function
  def cubic_kernel(t, a=-0.5):
    """
    Cubic kernel function.
    t: float or array[float]
    a: float
    """

    # Compute the absolute values of t, t^2 and t^3
    abs_t = np.abs(t)
    abs_t2 = abs_t**2
    abs_t3 = abs_t**3

    # Apply the correct formula for t being within the range of 0-1 or 1-2
    result = np.where(
        abs_t <= 1,
        (a+2)*abs_t3 - (a+3)*abs_t2 + 1,
        np.where(
            abs_t < 2,
            a*abs_t3 - 5*a*abs_t2 + 8*a*abs_t - 4*a,
            0
        )
    )

    return result

  # Define the bicubic interpolation function
  def bicubic_interpolation(image, src_y, src_x):
    """
    Two-layered bicubic interpolation function.
    image: 2D numpy array (grayscale)
    src_x: float
    src_y: float
    """

    # Find the pixel to the top left in the original image
    y0 = int(np.floor(src_y))
    x0 = int(np.floor(src_x))

    # Compute the horizontal and vertical distances (4x4 grid)
    dx = src_x - ((x0 - 1) + np.arange(4))
    dy = src_y - ((y0 - 1) + np.arange(4))

    # Compute the horizontal and vertical weights using cubic kernel
    wx = cubic_kernel(dx)
    wy = cubic_kernel(dy)

    # Restrict the 4x4 grid to image bounds
    pixel_values = np.zeros((4,4))
    for j in range(4):
        for i in range(4):
            y = np.clip(y0 - 1 + j, 0, image.shape[0] - 1)
            x = np.clip(x0 - 1 + i, 0, image.shape[1] - 1)
            pixel_values[j,i] = image[y, x]

    # Apply bicubic interpolation along the x-axis
    column_values = np.dot(pixel_values, wx)

    # Apply bicubic interpolation along the y-axis
    pixel_value = np.dot(column_values, wy)

    return pixel_value

  # Run upsampling using bicubic interpolation
  return upsample(image, scale, bicubic_interpolation)


In [84]:
test_img = np.array([
    [ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100],
    [ 15,  25,  35,  45,  55,  65,  75,  85,  95, 105],
    [ 20,  30,  40,  50,  60,  70,  80,  90, 100, 110],
    [ 25,  35,  45,  55,  65,  75,  85,  95, 105, 115],
    [ 30,  40,  50,  60,  70,  80,  90, 100, 110, 120],
    [ 35,  45,  55,  65,  75,  85,  95, 105, 115, 125],
    [ 40,  50,  60,  70,  80,  90, 100, 110, 120, 130],
    [ 45,  55,  65,  75,  85,  95, 105, 115, 125, 135],
    [ 50,  60,  70,  80,  90, 100, 110, 120, 130, 140],
    [ 55,  65,  75,  85,  95, 105, 115, 125, 135, 145]
], dtype=np.uint8)

test_img_3x3 = np.array([
    [10, 20, 30],
    [15, 25, 35],
    [20, 30, 40]
], dtype=np.uint8)

In [88]:
def test_max_pooling():
    print("Test 1: Max Pooling Downsample (2x2, stride 2)")
    out = max_pooling_downsample(test_img, pool_size=(2,2), stride=(2,2))
    print(out, "\n")

def test_avg_pooling():
    print("Test 2: Average Pooling Downsample (2x2, stride 2)")
    out = avg_pooling_downsample(test_img, pool_size=(2,2), stride=(2,2))
    print(out, "\n")

def test_med_pooling():
    print("Test 3: Median Pooling Downsample (2x2, stride 2)")
    out = med_pooling_downsample(test_img, pool_size=(2,2), stride=(2,2))
    print(out, "\n")

def test_nn_upsample():
    print("Test 4: Nearest Neighbor Upsample (scale=2x2)")
    out = nn_interpolation_upsample(test_img, scale=(2,2))
    print(out, "\n")

def test_bilinear_upsample():
    print("Test 5: Bilinear Upsample (scale=2x2)")
    out = bilinear_interpolation_upsample(test_img, scale=(2,2))
    print(out, "\n")

def test_bicubic_upsample():
    print("Test 6: Bicubic Upsample (scale=2x2)")
    out = bicubic_interpolation_upsample(test_img, scale=(2,2))
    print(out, "\n")


test_max_pooling()
test_avg_pooling()
test_med_pooling()
test_nn_upsample()
test_bilinear_upsample()
test_bicubic_upsample()

Test 1: Max Pooling Downsample (2x2, stride 2)
[[ 25  45  65  85 105]
 [ 35  55  75  95 115]
 [ 45  65  85 105 125]
 [ 55  75  95 115 135]
 [ 65  85 105 125 145]] 

Test 2: Average Pooling Downsample (2x2, stride 2)
[[ 17  37  57  77  97]
 [ 27  47  67  87 107]
 [ 37  57  77  97 117]
 [ 47  67  87 107 127]
 [ 57  77  97 117 137]] 

Test 3: Median Pooling Downsample (2x2, stride 2)
[[ 17  37  57  77  97]
 [ 27  47  67  87 107]
 [ 37  57  77  97 117]
 [ 47  67  87 107 127]
 [ 57  77  97 117 137]] 

Test 4: Nearest Neighbor Upsample (scale=2x2)
[[ 10  20  20  30  30  40  40  50  50  60  60  70  70  80  80  90  90 100
  100 100]
 [ 15  25  25  35  35  45  45  55  55  65  65  75  75  85  85  95  95 105
  105 105]
 [ 15  25  25  35  35  45  45  55  55  65  65  75  75  85  85  95  95 105
  105 105]
 [ 20  30  30  40  40  50  50  60  60  70  70  80  80  90  90 100 100 110
  110 110]
 [ 20  30  30  40  40  50  50  60  60  70  70  80  80  90  90 100 100 110
  110 110]
 [ 25  35  35  45  45  55  